# ProjectManager, PipelineManager and TaskManager
Within the task hierarchy we can use the ProjectManager, PipelineManager and TaskManager to run a set of tasks and agents. They are all abstractions from BaseTaskManager which provides a communication system and the default execution logic. Regardless of the hierarchy each manager is assigned to a task.

## Startup sequence
Every execution run starts with a startup sequence, where the manager will verify it's resources, wait for task approval, fetch and resolve issues (if an issue handler is provided), fetch feedback from user or other agents, that will be added to the input prompt for the executing agent.
## Execution sequence
The execution sequence is the main loop of the manager, where it will execute the agents and tasks in the order they are provided. A Project Manager will initiate and run a Pipeline Manager for each pipeline in the project. Each Pipeline Manager will initiate and run a Task Manager for each task in the pipeline. Each Task Manager will initiate and run an Agent Executor and try to complete the task.
The execution process returns either the agent final or an issue that may be due to an error or reported by the agent itself.

## Shutdown sequence
The shutdown sequence will run after the execution sequence is completed. The shutdown sequence is responsible for verifying the results and updating the task status. If an issue is returned by the execution agent it will be reported to higher level managers. Basiclly the next higher manager with an issue handler will be responsible for resolving the issue.

### !Note on Issues:
To lay a fundament for a robust execution system we have implemented an issue system. An issue is a message that is passed between managers and agents. At the current state you can only use the simple issue handler which will just create a task for the human user to resolve. In the future we will add agents as issue handlers that can try to resolve the issue in the first place.


# Running a PipelineManager
From our previous example "Demographic Report India" we will run a pipeline manager for executing the pipeline.
To use the pipeline manager we need to create Agents first and register them in the AgentRegistry. We register the agents with the AgentConfig, which is a simple BaseModel that holds the agent name and the agent_type and its tools or toolkit.

### Defining Agents
Mandatory fields for an AgentConfig are: name, description
It should also have set either a list of tools or toolkit name. If both are set the toolkit will be loaded and the tools will be added to the toolkit.
To report issues it is recommended to add "ReportIssue" tool to the agent.
If no agent_type is provided the default AgentType will be ZeroShotAgent. But you can use any other agent_type from langchain. As well as define custom AgentTypes.
**Note: Custom Agent Types not implemented yet**


In [1]:
from task_chain.agents.agent_registry import AgentRegistry, AgentConfig
from task_chain.tools.report_issue import IssueReportTool
from langchain import SerpAPIWrapper
from langchain.tools import Tool

def fake_analyze_tool(data: str):
    return f"Analyzed: {data} successfully"

def fake_report_tool(data: str):
    return f"Created report: {data} successfully"

AGENTS = [
    AgentConfig(
        name="ResearchAgent",
        description="Agent for researching data from web",
        tools=[
            Tool(name="web_search", func=SerpAPIWrapper().run, description="useful for searching the web for information. Input is  search query"),
            IssueReportTool()
        ]
    ),
    AgentConfig(
        name="DataAnalyzingAgent",
        description="Agent for analyzing data",
        tools=[
            Tool(name="report", func=fake_analyze_tool, description="useful for analyzing data. Takes a string containing data as input"),
            IssueReportTool()
        ]
    ),
    AgentConfig(
        name="ReportAgent",
        description="Agent for creating reports",
        tools=[
            Tool(name="report", func=fake_report_tool, description="useful for creating reports. Takes a string containing analyzed data as input"),
            IssueReportTool()
        ]
    )
]


agent_registry = AgentRegistry(agents=AGENTS)

Now we can run pipeline decomposer to create a pipeline of tasks. This time we will use AgentRegistry to assign and load agents to the tasks.

In [2]:
from task_chain.decompose.pipeline import SimplePipelineDecomposer
from task_chain.storage.storage_context import TaskContextStore

task_storage = TaskContextStore()
decomposer = SimplePipelineDecomposer(task_storage, agents=agent_registry.get_all())

objective = "Create a report about the demographics of the population of India"

decomposer.create_tasks(objective)





> Entering new LLMChain chain...
Prompt after formatting:

Breakdown the following task into a sequence of subtasks. Define the input and output keys of each task as lowercase strings.
ROOT TASK: 
Create a report about the demographics of the population of India


Return your respond as string in the following format without any introducing or describing text:

Name of Root Task:
    - Name of Subtask 1
    - Name of Subtask 2
    ...
    - Name of Subtask n
    


Additional instructions:
Create a flat list as shown above do not break subtasks down further.
Try to break down into as few subtasks as possible.
Begin!


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Based on the following sequence of subtasks, define the input and output keys of each task as lowercase strings with underscores and
add a description for each subtask.
TASK LIST: 
root task:
    - gather demographic data on India's population
    - analyze demographic data to identify key tr

(Task(id='278fe5b43b4b4ba69149189e4d519ba3', name="Create report on demographics of India's population", description="Compile and summarize the demographic data and key findings for India's population, including trends and patterns identified through statistical analysis and visual aids.", status=<TaskStatus.OPEN: 'open'>, type=<TaskType.PIPELINE: 'pipeline'>, summary="root task:\n    - gather demographic data on India's population\n    - analyze demographic data to identify key trends and patterns\n    - create visual aids to present demographic data and trends\n    - write report summarizing demographic data and key findings for India's population. \n\ninput key: none\noutput key: report on demographics of India's population", details=None, results=None, relations={}, inputs=['demographic_data_india', 'demographic_trends_india', 'demographic_visual_aids'], outputs=['report_on_demographics_india']),
 [Task(id='88b3dedf35b341d8911e86f33f990521', name="gather demographic data on India's

Next we create a pipeline manager and run it.
We have to use a NonInteractiveCommunicator as our communication system for this example, because we cannot run the current implementation of console interactions in jupyter.

In [11]:
task_storage.task_network.root_tasks.pop(0)

'579d1dc514c1456c9ad9ead5dd91f013'

In [3]:
from task_chain.executor.pipeline import PipelineManager
from task_chain.communication.non_interactive import NonInteractiveCommunicator
from task_chain.executor.issue_handler.simple import SimpleIssueHandler

communicator = NonInteractiveCommunicator()
issue_handler = SimpleIssueHandler(task_storage)

pipeline_manager = PipelineManager(
    task=task_storage.get_task(task_storage.root_id),
    agent_registry=agent_registry,
    issue_handler=issue_handler,
    communication=communicator,
    task_storage=task_storage,
    persist=True,
    persist_path="./task_storage.json",
    verbose=True
)

In [4]:
pipeline_manager.execute()



> Entering new PipelineManager chain...

CURRENT CONTEXT:
┐
└──Create report on demographics of India's population <<<<<<<<
   ├──gather demographic data on India's population
   ├──analyze demographic data to identify key trends and patterns
   ├──create visual aids to present demographic data and trends
   └──write report summarizing demographic data and key findings for India's population

starting:

 --------------------------------------------------------------------
| TASK TYPE:    PIPELINE                            STATUS: OPEN     |
|                                                                    |
| ID:           278fe5b43b4b4ba69149189e4d519ba3                     |
|                                                                    |
| NAME:         Create report on demographics of India's populati-   |
|               on                                                   |
| DESCRIPTION:  Compile and summarize the demographic data and ke-   |
|               y findin

Task(id='278fe5b43b4b4ba69149189e4d519ba3', name="Create report on demographics of India's population", description="Compile and summarize the demographic data and key findings for India's population, including trends and patterns identified through statistical analysis and visual aids.", status=<TaskStatus.ISSUE: 'issue'>, type=<TaskType.PIPELINE: 'pipeline'>, summary="root task:\n    - gather demographic data on India's population\n    - analyze demographic data to identify key trends and patterns\n    - create visual aids to present demographic data and trends\n    - write report summarizing demographic data and key findings for India's population. \n\ninput key: none\noutput key: report on demographics of India's population", details=None, results=None, relations={<TaskRelations.ISSUE: 'issue'>: 'd7852e464c794c22bf80e717cba1513f'}, inputs=['demographic_data_india', 'demographic_trends_india', 'demographic_visual_aids'], outputs=['report_on_demographics_india'])

In this run we see an error while using SerpAPI Tool. We can inpect

In [15]:
pipeline_manager.execute()



> Entering new PipelineManager chain...

CURRENT CONTEXT:
┐
└──Create report on demographics of India's population <<<<<<<<
   ├──gather demographic data on India's population
   │  └──[ISSUE]: undefined_issue in task gather demographic data on India's population
   ├──analyze demographic data to identify key trends and patterns
   ├──create visual aids to present demographic data and trends
   └──write report summarizing demographic data and key findings for India's population

starting:

 --------------------------------------------------------------------
| TASK TYPE:    PIPELINE                           STATUS: ISSUE     |
|                                                                    |
| ID:           278fe5b43b4b4ba69149189e4d519ba3                     |
|                                                                    |
| NAME:         Create report on demographics of India's populati-   |
|               on                                                   |
| DESCR

Task(id='278fe5b43b4b4ba69149189e4d519ba3', name="Create report on demographics of India's population", description="Compile and summarize the demographic data and key findings for India's population, including trends and patterns identified through statistical analysis and visual aids.", status=<TaskStatus.ISSUE: 'issue'>, type=<TaskType.PIPELINE: 'pipeline'>, summary="root task:\n    - gather demographic data on India's population\n    - analyze demographic data to identify key trends and patterns\n    - create visual aids to present demographic data and trends\n    - write report summarizing demographic data and key findings for India's population. \n\ninput key: none\noutput key: report on demographics of India's population", details=None, results=None, relations={<TaskRelations.ISSUE: 'issue'>: 'f9028b14da5b4c4cb866817e17c2379f'}, inputs=['demographic_data_india', 'demographic_trends_india', 'demographic_visual_aids'], outputs=['report_on_demographics_india'])

In [17]:
for task in task_storage.task_store.get_all().values():
    print(task.colored_card_str())

 --------------------------------------------------------------------
| TASK TYPE:    PIPELINE                           STATUS: ISSUE     |
|                                                                    |
| ID:           278fe5b43b4b4ba69149189e4d519ba3                     |
|                                                                    |
| NAME:         Create report on demographics of India's populati-   |
|               on                                                   |
| DESCRIPTION:  Compile and summarize the demographic data and ke-   |
|               y findings for India's population, including tren-   |
|               ds and patterns identified through statistical an-   |
|               alysis and visual aids.                              |
| INPUTS:       demographic_data_india                               |
|               demographic_trends_india                             |
|               demographic_visual_aids                              |
| OUTPU